In [1]:
import pandas as pd
import numpy as np

In [491]:
def readBlosum62():
    '''
    Read scoring matrix
    '''
    with open('blosum62.txt') as matrix_file:
        matrix = matrix_file.read()
        lines = matrix.strip().split('\n')

    blosum={}
    cols = lines[0].split()

    for row in lines[1:]:       
        idx = row[0]
        vals = row[1:].split()
        vals = [int(_) for _ in vals]
        #print(idx,vals)
        blosum[idx]=vals

    blosum = pd.DataFrame.from_dict(blosum,orient='index',columns=cols)
    return blosum

def scorematch(residueA,residueB,scoringMatrix):
    '''
    Get the value of a score between two matches
    '''
    return scoringMatrix.loc[residueA][residueB]

def createScorMat(match = 1,mismatch = 0):
    """
    Create scoring matrix for nucleotides with a given match and mismatch values
    """
    nts = ['A','C','T','G']

    scoringMatrix = np.zeros((len(nts),len(nts)),)
    np.fill_diagonal(scoringMatrix,match)
    scoringMatrix
    scoringMatrix[scoringMatrix == 0] = mismatch
    scoringMatrix = pd.DataFrame(scoringMatrix,index=nts,columns=nts)
    return scoringMatrix

In [3]:
def linearSpaceAlign(x,y,scoringMatrix,indel,verbose=False):
    ####
    m = len(y)
    n = len(x)
    score = list(np.cumsum([indel for _ in y]))
    nullScore = list(np.cumsum([0] + [indel for _ in x]))
    prev = score.copy()
    score,nullScore,prev
    ####
    for j,residue in enumerate(x):
        if verbose: print(j,residue)
        score = [nullScore[j]] + score
        prev = score.copy()

        for i in range(1,m+1):
            if residue == y[i-1]:
                if verbose: print(i, residue,'match',y[i-1], prev,score)
                score[i] = prev[i-1] + scorematch(residue,y[i-1],scoringMatrix) 
            else:
                if verbose: print(i, residue, 'no match', y[i-1], prev,score,'choose',
                                      score[i-1],
                                      score[i],prev[i-1])
                score[i] = max(score[i-1] + indel, #Up
                                      score[i] + indel, #Left
                                      prev[i-1] + scorematch(residue,y[i-1],scoringMatrix)) #Mismatch
            if verbose: print(score,"\n----")

        score = score[1:]
    
    score = [nullScore[j+1]] + score
    return score


In [352]:
##### Backtracking        
def scoreAlign(x,y,scoringMatrix,indel):
    m = len(x)
    n = len(y)
    Fmatrix = np.zeros((m+1,n+1))

    ## initialize
    for i in range(1,m+1):
        Fmatrix[i,0] += Fmatrix[i-1,0] + indel
    for j in range(1,n+1):
        Fmatrix[0,j] += Fmatrix[0,j-1] + indel
    ##
    for i in range(1,m+1):
        for j in range(1,n+1):
            if x[i-1] == y[j-1]:
                Fmatrix[i,j] = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1],scoringMatrix)
            else:
                mm = Fmatrix[i-1,j-1] + scorematch(x[i-1],y[j-1],scoringMatrix)
                down = Fmatrix[i-1,j] + indel
                right = Fmatrix[i,j-1] + indel
                Fmatrix[i,j] = max([mm,down,right])
    
    return Fmatrix,Fmatrix[m][n]

def backtrack(x,y,Fmatrix,scoringMatrix,indel,verbose=False):
    """
    """
    i = len(x)
    j = len(y)

    wOut = ''
    vOut = ''

    while i*j != 0:
        if verbose: print(i,j,x[i-1],y[j-1],"")
        if x[i-1] == y[j-1]:
            if verbose: print(x[i-1],y[j-1])
            wOut += x[i-1]
            vOut += y[j-1]
            i -= 1
            j -= 1
        else:
            btrack = [Fmatrix[i][j-1] + indel,  #up
                            Fmatrix[i-1][j] + indel,   #left
                            Fmatrix[i-1][j-1] + scorematch(x[i-1],y[j-1],scoringMatrix)]
            mx = np.argmax(btrack) #diag

            if verbose: print(i,j,btrack,mx,btrack[mx])

            if mx == 2:
                wOut += x[i-1]
                vOut += y[j-1]
                i -= 1
                j -= 1
            else:
                if mx == 0:
                    wOut += '-'
                    vOut += y[j-1]
                    j -= 1
                else:
                    wOut += x[i-1]
                    vOut += '-'
                    i -= 1
        if verbose: print(wOut[::-1])
        if verbose: print(vOut[::-1])
        if verbose: print("~~~~\n")
    
    if verbose: print('exit status',i,j)
    if j == 0 and i>0:
        if verbose: print('i=0')
        wOut = wOut + x[i-1::-1]
        wOut = wOut[::-1]
        vOut = ''.join(['-' for _ in range(i,0,-1)]) + vOut[::-1]
    if i == 0 and j>0:
        if verbose: print('j=0')
        vOut = vOut + y[j-1::-1]
        vOut = vOut[::-1]
        wOut = ''.join(['-' for _ in range(j,0,-1)]) + wOut[::-1]
    if j == 0 and i==0:
        if verbose: print('reverse all')
        wOut = wOut[::-1]
        vOut = vOut[::-1]
        
    return vOut, wOut

def nwAlign(x,y,scoringMatrix,indel,verbose=False):
    """
    Peforms Needleman-Wunsch alignment.
    Inefficient for long strings, takes O(m*n)
    """
    fmat, pts = scoreAlign(x,y,scoringMatrix,indel)
    if verbose: print('backtracking')
    z,w = backtrack(x,y,fmat,scoringMatrix,indel,verbose)
    return z,w,pts,fmat


In [332]:
# fm,_ = scoreAlign('AGTACGCA','TATGC',scoringMatrix,-2)
# backtrack('AGTACGCA','TATGC',fm,scoringMatrix,-2,verbose=True)

In [337]:
nwAlign('C','GGG',scoringMatrix,-2,verbose=False)

reverse all


('GGG',
 'C--',
 -5.0,
 array([[ 0., -2., -4., -6.],
        [-2., -1., -3., -5.]]))

In [338]:
def splitstring(s,point):
    A = s[0:point:]
    B = s[point:]#[::-1]
    return A, B

def halfstring(s):
    A = s[0:int(len(s)/2):]
    B = s[int(len(s)/2):] #[::-1]
    return A, B

In [339]:
x='TATGC'
y='AGTACGCA'
####
indel = -2
mismatch=-1
match = 2
### Create scoring matrix
scoringMatrix = createScorMat(match,mismatch)
###
linearSpaceAlign(y,x,scoringMatrix,indel)

[-16, -12.0, -8.0, -7.0, -3.0, 1.0]

In [342]:
nwAlign(y,x,scoringMatrix,indel)

('--TATGC-',
 'AGTACGCA',
 1.0,
 array([[  0.,  -2.,  -4.,  -6.,  -8., -10.],
        [ -2.,  -1.,   0.,  -2.,  -4.,  -6.],
        [ -4.,  -3.,  -2.,  -1.,   0.,  -2.],
        [ -6.,  -2.,  -4.,   0.,  -2.,  -1.],
        [ -8.,  -4.,   0.,  -2.,  -1.,  -3.],
        [-10.,  -6.,  -2.,  -1.,  -3.,   1.],
        [-12.,  -8.,  -4.,  -3.,   1.,  -1.],
        [-14., -10.,  -6.,  -5.,  -1.,   3.],
        [-16., -12.,  -8.,  -7.,  -3.,   1.]]))

In [343]:
yA,yB = halfstring(y)
scoreF = linearSpaceAlign(yA,x,scoringMatrix,indel)
scoreR = linearSpaceAlign(yB[::-1],x[::-1],scoringMatrix,indel)
print (scoreF,scoreR)
print (np.array(scoreF) + np.array(scoreR[::-1]))

[-8, -4.0, 0.0, -2.0, -1.0, -3.0] [-8, -4.0, 0.0, 1.0, -1.0, -3.0]
[-11.  -5.   1.  -2.  -5. -11.]


In [344]:
#### Book example
x='ACGGAA'
y='ATTCAA'
####
indel = 0
mismatch= 0
match = 1
### Create scoring matrix
scoringMatrix = createScorMat(match,mismatch)
print(scoringMatrix)
###
linearSpaceAlign(x,y,scoringMatrix,indel)


     A    C    T    G
A  1.0  0.0  0.0  0.0
C  0.0  1.0  0.0  0.0
T  0.0  0.0  1.0  0.0
G  0.0  0.0  0.0  1.0


[0, 1.0, 1.0, 1.0, 2.0, 3.0, 4.0]

In [345]:
xA,xB = halfstring(x)
print(f,r)
scoreF = linearSpaceAlign(xA,y,scoringMatrix,indel)
scoreR = linearSpaceAlign(xB[::-1],y[::-1],scoringMatrix,indel)

arrSum = np.array(scoreF) + np.array(scoreR[::-1])
maxScore = max(arrSum)
whichMax = np.argmax(arrSum)
print ("",scoreF,"\n",scoreR[::-1],"\n",arrSum,"\n",
       'alignment score', maxScore,
       '\n position',whichMax)

ACG GAA
 [0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0] 
 [2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 0] 
 [2. 3. 3. 3. 4. 3. 2.] 
 alignment score 4.0 
 position 4


In [475]:
nwAlign(x,y,scoringMatrix,indel)

('AGTACGCA',
 '--TATGC-',
 1.0,
 array([[  0.,  -2.,  -4.,  -6.,  -8., -10., -12., -14., -16.],
        [ -2.,  -1.,  -3.,  -2.,  -4.,  -6.,  -8., -10., -12.],
        [ -4.,   0.,  -2.,  -4.,   0.,  -2.,  -4.,  -6.,  -8.],
        [ -6.,  -2.,  -1.,   0.,  -2.,  -1.,  -3.,  -5.,  -7.],
        [ -8.,  -4.,   0.,  -2.,  -1.,  -3.,   1.,  -1.,  -3.],
        [-10.,  -6.,  -2.,  -1.,  -3.,   1.,  -1.,   3.,   1.]]))

A faster approach is to split the query in half and perform two searches: 
1) The first half of the query against the target string
2) The reversed second half of the query against the reversed target.

The resulting scoring vectors are then summed (the second vector has to be reversed). The highest number is the total score. The index is the column or row through which the backtracking algorithm would pass.

In [354]:
#### https://gtuckerkellogg.github.io/pairwise/demo/
blosum = readBlosum62()
x='HEAGAWGHEE'
y='PAWHEAE'
indel=-5
####
###
nwAlign(y,x,blosum,indel,verbose=False)

('HEAGAWGHE-E',
 '-P--AW-HEAE',
 7.0,
 array([[  0.,  -5., -10., -15., -20., -25., -30., -35., -40., -45., -50.],
        [ -5.,  -2.,  -6., -11., -16., -21., -26., -31., -36., -41., -46.],
        [-10.,  -7.,  -3.,  -2.,  -7., -12., -17., -22., -27., -32., -37.],
        [-15., -12.,  -8.,  -6.,  -4.,  -9.,  -1.,  -6., -11., -16., -21.],
        [-20.,  -7., -12., -10.,  -8.,  -6.,  -6.,  -3.,   2.,  -3.,  -8.],
        [-25., -12.,  -2.,  -7., -12.,  -9.,  -9.,  -8.,  -3.,   7.,   2.],
        [-30., -17.,  -7.,   2.,  -3.,  -8., -12.,  -9.,  -8.,   2.,   6.],
        [-35., -22., -12.,  -3.,   0.,  -4.,  -9., -14.,  -9.,  -3.,   7.]]))

In [346]:
#### https://gtuckerkellogg.github.io/pairwise/demo/
blosum = readBlosum62()
x='HEAGAWGHEE'
y='PAWHEAE'
indel=-5
####
###
linearSpaceAlign(x,y,blosum,indel)

[-50, -46, -37, -21, -8, 2, 6, 7]

In [347]:
xA,xB = halfstring(x)
print(f,r)
scoreF = linearSpaceAlign(xA,y,blosum,indel)
scoreR = linearSpaceAlign(xB[::-1],y[::-1],blosum,indel)

arrSum = np.array(scoreF) + np.array(scoreR[::-1])
maxScore = max(arrSum)
whichMax = np.argmax(arrSum)
print ("",scoreF,"\n",scoreR[::-1],"\n",arrSum,"\n",
       'alignment score', maxScore,
       '\n position',whichMax)

ACG GAA
 [-25, -21, -12, -9, -6, -9, -8, -4] 
 [9, 14, 19, 3, -6, -10, -15, -25] 
 [-16  -7   7  -6 -12 -19 -23 -29] 
 alignment score 7 
 position 2


### Wikipedia example

In [355]:
x='TATGC'
y='AGTACGCA'
####
indel = -2
mismatch= -1
match = 2
### Create scoring matrix
scoringMatrix = createScorMat(match,mismatch)
print(scoringMatrix)
###
linearSpaceAlign(x,y,scoringMatrix,indel)

     A    C    T    G
A  2.0 -1.0 -1.0 -1.0
C -1.0  2.0 -1.0 -1.0
T -1.0 -1.0  2.0 -1.0
G -1.0 -1.0 -1.0  2.0


[-10, -6.0, -2.0, -1.0, -3.0, 1.0, -1.0, 3.0, 1.0]

In [357]:
nwAlign('TATGC','AGTACGCA',scoringMatrix,-2)

('AGTACGCA',
 '--TATGC-',
 1.0,
 array([[  0.,  -2.,  -4.,  -6.,  -8., -10., -12., -14., -16.],
        [ -2.,  -1.,  -3.,  -2.,  -4.,  -6.,  -8., -10., -12.],
        [ -4.,   0.,  -2.,  -4.,   0.,  -2.,  -4.,  -6.,  -8.],
        [ -6.,  -2.,  -1.,   0.,  -2.,  -1.,  -3.,  -5.,  -7.],
        [ -8.,  -4.,   0.,  -2.,  -1.,  -3.,   1.,  -1.,  -3.],
        [-10.,  -6.,  -2.,  -1.,  -3.,   1.,  -1.,   3.,   1.]]))

In [360]:
def Hirschberg(x,y,scoringMatrix,indel,verbose=False):           
    xA,xB = halfstring(x)
    if verbose: print(xA,xB)
    scoreF = linearSpaceAlign(xA,y,scoringMatrix,indel)
    scoreR = linearSpaceAlign(xB[::-1],y[::-1],scoringMatrix,indel)

    arrSum = np.array(scoreF) + np.array(scoreR[::-1])
    maxScore = max(arrSum)
    whichMax = np.argmax(arrSum)
    if verbose:
        print ("",scoreF,"\n",scoreR[::-1],"\n",arrSum,"\n",
           'alignment score:', maxScore,
           '\n position',whichMax)
    yA, yB = splitstring(y,whichMax)
    return xA,xB,yA,yB,

In [396]:
recursiveAlign("CTAG","A")

('--A-', 'CTAG')

In [393]:
def recursiveAlign(x,y):           
    z = '' #X alignment
    w = '' #Y alignment
    
    if len(x) == 0 and len(y)>0:
        for i in range(0,len(y)-1):
            z = z + '-'
            w = w + y[i]
            
    if len(y) == 0 and len(x)>0:
        for i in range(0,len(x)-1):
            w = w + '-'
            z = z + x[i]
            
    if len(y) == 1 or len(x) == 1: 
        tmpZ,tmpW,_,fm = nwAlign(x,y,scoringMatrix,indel)
        z += tmpZ
        w += tmpW
    else:
        Hirschberg('TA','AGTA',scoringMatrix,indel)
        

    return z,w

In [374]:
### Full call, divides x by half and y into middle column
xF,xR,yA,yB, maxScore = Hirschberg(x,y,scoringMatrix,indel)
xF,xR,yA,yB, maxScore

('TA', 'TGC', 'AGTA', 'CGCA', 1.0)

In [467]:
recHirschberg('TATGC','AGTACGCA',scoringMatrix,indel,v=True)

TATGC AGTACGCA
recursive
TA TGC
 [-4, 0.0, -2.0, -4.0, 0.0, -2.0, -4.0, -6.0, -8.0] 
 [-4.0, -2.0, 0.0, -1.0, 1.0, 0.0, -4.0, -5.0, -6] 
 [ -8.  -2.  -2.  -5.   1.  -2.  -8. -11. -14.] 
 alignment score: 1.0 
 position 4
split TA AGTA TGC CGCA


('AGTACGCA', '--TATGC-', 1.0)

In [ ]:
### Left path

In [ ]:
### Right path

In [375]:
Hirschberg('TA','AGTA',scoringMatrix,indel)

('T', 'A', 'AGT', 'A', 0.0)

In [167]:
Hirschberg('TGC','CGCA',scoringMatrix,indel)

('T', 'GC', 'C', 'GCA')

In [377]:
nwAlign('T','AGT',scoringMatrix,indel)

('AGT',
 '--T',
 -2.0,
 array([[ 0., -2., -4., -6.],
        [-2., -1., -3., -2.]]))

In [380]:
nwAlign('T','C',scoringMatrix,indel)

('C',
 'T',
 -1.0,
 array([[ 0., -2.],
        [-2., -1.]]))

In [170]:
nwAlign('A','A',indel,scoringMatrix)

('A', 'A', 2.0)

In [381]:
Hirschberg('GC','GCA',scoringMatrix,indel)

('G', 'C', 'G', 'CA', 2.0)

In [176]:
""

''

In [383]:
nwAlign('G','G',scoringMatrix,indel)

('G',
 'G',
 2.0,
 array([[ 0., -2.],
        [-2.,  2.]]))

In [392]:
nwAlign('C','CA',scoringMatrix,indel)

('CA',
 'C-',
 0.0,
 array([[ 0., -2., -4.],
        [-2.,  2.,  0.]]))

In [482]:
def recHirschberg(x,y,scoringMatrix,indel,v=False,verbose=False):
    maxScore = 0  
    z = '' #X alignment
    w = '' #Y alignment
    if v: print(x,y)
       
    #     if len(x) == 0 and len(y)>0:
    #         print('x=0')
    #         for i in range(0,len(y)-1):
    #             w = w + '-'
    #             z = z + y[i]

    #     if len(y) == 0 and len(x)>0:
    #         print('y=0')
    #         for i in range(0,len(x)-1):
    #             z = z + '-'
    #             w = w + x[i]
            
    if len(y) == 1 or len(x) == 1: 
        if v: print('one is 1',x,y)
        tmpZ,tmpW,_,fm = nwAlign(x,y,scoringMatrix,indel,verbose=False)
        z += tmpZ
        w += tmpW
    else:
        if v: print('recursive')
        xA,xB = halfstring(x)
         
        ### Determine the split point 
        scoreF = linearSpaceAlign(xA,y,scoringMatrix,indel)
        scoreR = linearSpaceAlign(xB[::-1],y[::-1],scoringMatrix,indel)
        ### Get the max
        arrSum = np.array(scoreF) + np.array(scoreR[::-1])
        maxScore = max(arrSum)
        whichMax = np.argmax(arrSum)
        
        
        if v: print ("",scoreF,"\n",scoreR[::-1],"\n",arrSum,"\n",
               'alignment score:', maxScore,
               '\n position',whichMax)
        
        yA, yB = splitstring(y,whichMax)        
        if v: print('split',xA,yA,xB,yB)    
        tmpZl,tmpWl,_ = recHirschberg(xA,yA,scoringMatrix,indel)
        tmpZr,tmpWr,_ = recHirschberg(xB,yB,scoringMatrix,indel)
        z += tmpZl + tmpZr
        w += tmpWl + tmpWr
    return z,w,maxScore

In [483]:
recHirschberg('AGTACGCA','TATGC',scoringMatrix,indel)

('--TA-TGC-', 'AGTAC-GCA', 4.0)

---

In [484]:
x='ACGGAA'
y='ATTCAA'
####
indel = 0
mismatch= 0
match = 1
scoringMatrix = createScorMat(match,mismatch)

In [487]:
%time
#### Needleman-Wunsch
z,w,s,_ = nwAlign(x,y,scoringMatrix,indel)
print(s)
print(w)
print(z)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
4.0
A--CGGAA
ATTC--AA


In [488]:
### Recursive solution
%time
z,w,s = recHirschberg(x,y,scoringMatrix,indel)
print(s)
print(w)
print(z)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
4.0
A--CGGAA
ATTC--AA


In [489]:
x = 'ILYPRQSMICMSFCFWDMWKKDVPVVLMMFLERRQMQSVFSWLVTVKTDCGKGIYNHRKYLGLPTMTAGDWHWIKKQNDPHEWFQGRLETAWLHSTFLYWKYFECDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIWMATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKNYMMAPVVNDPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMNTDPHYCQTHVCWISAMFPIYSPFYTSGPKEFYMWLQARIDQNMHGHANHYVTSGNWDSVYTPEKRAGVFPVVVPVWYPPQMCNDYIKLTYECERFHVEGTFGCNRWDLGCRRYIIFQCPYCDTMKICYVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEWCNQPVLAKFVRSKIVAICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPTSHYVFMRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHHRRAFRAELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA'
y = 'ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVPLNPQMQTVFQNRGLDRVTVKTDCHDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLYKKTEYCLVRHSNCCHCDWDQIIHTCAFIAFLDLYQRHWPMCDKLYCHFHHSWFCRNQEMSMDWNQWFPWDSVPRANCLEEGALIALYAGIWANSMKRDMKTDHACTVRLIYVCELHAWLKYCYTSINMLCGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQGICPVERKSHCEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPPKEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSYKLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWACEAQTNVCERMLNNQHVAVKRNVTFWWQMVPNCLWSCHNKMTWPTRPEQHRLFFVKMRLKCMHEYLDVAPSDFCRNMQAYMHSMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGYIYDQSLQVVSCEGCSYYADCYVKCINVKEKCMFA'
indel = -5

In [ ]:
%time # takes too long
#### Needleman-Wunsch
# z,w,s,_ = nwAlign(x,y,blosum,indel)
# print(s)
# print(w)
# print(z)

In [490]:
### Recursive solution
%time
z,w,s = recHirschberg(x,y,blosum,indel)
print(s)
print(w)
print(z)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 23.1 µs
1555
ILYPRQSMICMSFCF-WDM--WKKDVPVVLMMFLERRQMQSVF-S-WL--VTVKTDCGKGIYNHR-K--Y-LGLPTMTAGDWHWIKK---Q-NDPHEWFQGRLETAWLHSTFLYWKYFE-CDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIW---MA----TDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKN-YMMAPVVNDPMIEA--FYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMN-TDPH----Y--CQTHVCW-ISAMF-PIYSPFYT--SG-PKEFYMW---LQARI-DQNM---HGHANHYV-TSGNWDSVYTPEKRA--G--V-FP-V-V-------VPVWYPPQMCN-DY-IKLTYEC--E---RFHVEGTFGCNRWD-L-GCRR--YII--FQCPYCDTMKI---CY--VDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEW-----CNQPVLAKFVRSKIVA---ICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPT--S-HYVF---MRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHH-RRAFRA----ELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA
ILIPRQQMGCFPFPWHFDFCFWSAHHSLVV-P-L-NPQMQTVFQNRGLDRVTVKTDC----HDHRWKWIYNLGLPTMTAGDWHFIKKH